<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 7

## NLP and Machine Learning on [travel.statsexchange.com](http://travel.stackexchange.com/) data

---

In Project 7 you'll be doing NLP and machine learning on post data from stackexchange's travel subdomain. 

This project is setup like a mini Kaggle competition. You are given the training data and when projects are submitted your model will be tested on the held-out testing data. There will be prizes for the people who build models that perform best on the held out test set!

---

## Notes on the data

The data is again compressed into the `.7z` file format to save space. There are 6 .csv files and one readme file that contains some information on the fields.

    posts_train.csv
    comments_train.csv
    users.csv
    badges.csv
    votes_train.csv
    tags.csv
    readme.txt
    
The data is located in your datasets folder:

    DSI-SF-2/datasets/stack_exchange_travel.7z
    
If you're interested in where this data came from and where to get more data from other stackexchange subdomains, see here:

https://ia800500.us.archive.org/22/items/stackexchange/readme.txt


### Recommended Utilities for .7z

- For OSX [Keka](http://www.kekaosx.com/en/) or [The Unarchiver](http://wakaba.c3.cx/s/apps/unarchiver.html). 
- For Windows [7-zip](http://www.7-zip.org/) is the standard. 
- For Linux try the `p7zip` utility.  `sudo apt-get install p7zip`.



<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 1. Use LDA to find what topics are discussed on travel.stackexchange.com.

---

Text can be found in the posts and the comments datasets. The `ParentId` column in the posts dataset indicates what the "question" post was for a given post. Comment text can be merged onto the post they are part of with the `PostId` field.

The text may have some HTML tags. BeautifulSoup has convenient ways to get rid of markup or extract text if you need to. You can also parse the strings yourself if you like.

The tags dataset has the "tags" that the users have officially given the post.

**1.1 Implement LDA against the text features of the dataset(s).**

- This can be posts or a combination of posts and comments if you want more power.
- Find optimal **K/num_topics**.

**1.2 Compare your topics to the tags. Do the LDA topics make sense? How do they compare to the tags?**


In [1]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import pandas as pd
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [178]:
travel_data = pd.read_csv('stack_exchange_travel/comments_train.csv')

In [179]:
travel_data1 = pd.read_csv('stack_exchange_travel/posts_train.csv')

In [180]:
travel_df = pd.DataFrame(travel_data)

In [181]:
travel_df1 = pd.DataFrame(travel_data1)

In [182]:
travel_df1.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,393.0,4.0,<p>My fiancée and I are looking for a good Car...,2013-02-25T23:52:47.953,4,NaN,2011-06-21T20:19:34.730,NaN,1,2012-05-24T14:52:14.760,...,NaN,101.0,NaN,9.0,NaN,1,8,<caribbean><cruising><vacations>,What are some Caribbean cruises for October?,361.0
1,NaN,1.0,<p>Singapore Airlines has an all-business clas...,NaN,1,NaN,2011-06-21T20:24:57.160,NaN,4,2013-01-09T09:55:22.743,...,NaN,693.0,NaN,24.0,NaN,1,8,<loyalty-programs><routes><ewr><singapore-airl...,Does Singapore Airlines offer any reward seats...,219.0
2,770.0,5.0,<p>Another definition question that interested...,NaN,0,NaN,2011-06-21T20:25:56.787,2.0,5,2012-10-12T20:49:08.110,...,NaN,101.0,NaN,13.0,NaN,1,11,<romania><transportation>,What is the easiest transportation to use thro...,340.0
3,62.0,3.0,<p>Can anyone suggest the best way to get from...,NaN,2,NaN,2011-06-21T20:30:38.687,1.0,8,2016-03-28T03:41:28.130,...,user141,NaN,NaN,26.0,NaN,1,11,<usa><airport-transfer><taxis><seattle>,Best way to get from SeaTac airport to Redmond?,9219.0
4,178.0,4.0,<p>We are considering visiting Argentina for u...,2016-01-02T10:26:48.277,1,NaN,2011-06-21T20:31:21.800,8.0,9,2016-01-01T21:58:02.303,...,NaN,101.0,NaN,23.0,NaN,1,12,<sightseeing><public-transport><transportation...,What are must-visit destinations for the first...,1503.0


In [185]:
total_travel_df = pd.concat([travel_df, travel_df1], axis=1, join='inner')
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(total_travel_df)

In [117]:
vectorizer.vocabulary_

{u'acceptedanswerid': 0,
 u'answercount': 1,
 u'body': 2,
 u'closeddate': 3,
 u'commentcount': 4,
 u'communityowneddate': 5,
 u'creationdate': 6,
 u'favoritecount': 7,
 u'id': 8,
 u'lastactivitydate': 9,
 u'lasteditdate': 10,
 u'lasteditordisplayname': 11,
 u'lasteditoruserid': 12,
 u'ownerdisplayname': 13,
 u'owneruserid': 14,
 u'parentid': 15,
 u'postid': 16,
 u'posttypeid': 17,
 u'score': 18,
 u'tags': 19,
 u'text': 20,
 u'title': 21,
 u'userdisplayname': 22,
 u'userid': 23,
 u'viewcount': 24}

In [186]:
docs = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
docs.head()

,acceptedanswerid,answercount,body,closeddate,commentcount,communityowneddate,creationdate,favoritecount,id,lastactivitydate,...,parentid,postid,posttypeid,score,tags,text,title,userdisplayname,userid,viewcount
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [119]:
vocab = {v: k for k, v in vectorizer.vocabulary_.iteritems()}
vocab

{0: u'acceptedanswerid',
 1: u'answercount',
 2: u'body',
 3: u'closeddate',
 4: u'commentcount',
 5: u'communityowneddate',
 6: u'creationdate',
 7: u'favoritecount',
 8: u'id',
 9: u'lastactivitydate',
 10: u'lasteditdate',
 11: u'lasteditordisplayname',
 12: u'lasteditoruserid',
 13: u'ownerdisplayname',
 14: u'owneruserid',
 15: u'parentid',
 16: u'postid',
 17: u'posttypeid',
 18: u'score',
 19: u'tags',
 20: u'text',
 21: u'title',
 22: u'userdisplayname',
 23: u'userid',
 24: u'viewcount'}

In [120]:
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [121]:
lda = models.LdaModel(
    matutils.Sparse2Corpus(X, documents_columns=False),
    # or use the corpus object created with the dictionary in the previous frame!
    # corpus, 
    num_topics  =  3,
    passes      =  20,
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)

In [122]:
frequency = defaultdict(int)

documents = total_travel_df

for text in documents:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in documents]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts)

# Create corpus matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [123]:
lda.print_topics(num_topics=3, num_words=7)

[(0,
  u'0.115*score + 0.115*id + 0.065*ownerdisplayname + 0.065*lasteditordisplayname + 0.065*favoritecount + 0.065*commentcount + 0.065*communityowneddate'),
 (1,
  u'0.127*creationdate + 0.072*answercount + 0.072*userdisplayname + 0.072*owneruserid + 0.072*acceptedanswerid + 0.072*title + 0.072*lasteditoruserid'),
 (2,
  u'0.092*tags + 0.092*lastactivitydate + 0.092*closeddate + 0.092*posttypeid + 0.092*text + 0.092*body + 0.023*parentid')]

In [124]:
lda.get_document_topics(corpus[1])

[(0, 0.42871581994060687), (1, 0.11272744051023298), (2, 0.4585567395491601)]

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 2. What makes an answer likely to be "accepted"?

---

**2.1 Build a model to predict whether a post will be marked as the answer.**

- This is a classification problem.
- You're free to use any of the machine learning algorithms or techniques we have learned in class to build the best model you can.
- NLP will be very useful here for pulling out useful and relevant features from the data. 
- Though not required, using bagging and boosting models like Random Forests and Gradient Boosted Trees will _probably_ get you the highest performance on the test data (but who knows!).


**2.2 Evaluate the performance of your classifier with a confusion matrix and accuracy. Explain how your model is performing.**

**2.3 Plot either a ROC curve or precision-recall curve (or both!) and explain what they tell you about your model.**

NOTE: You should only be predicting this for `PostTypeID=2` posts, which are the "answer" posts. This doesn't mean, however, that you can't or shouldn't use the parent questions as predictors!


In [204]:
total_travel_df.PostTypeId

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        2
10       2
11       2
12       1
13       1
14       1
15       1
16       2
17       1
18       2
19       1
20       1
21       2
22       2
23       2
24       2
25       2
26       2
27       2
28       1
29       2
        ..
41259    1
41260    2
41261    2
41262    2
41263    2
41264    1
41265    2
41266    1
41267    1
41268    2
41269    2
41270    2
41271    2
41272    2
41273    2
41274    5
41275    4
41276    1
41277    2
41278    2
41279    2
41280    2
41281    1
41282    2
41283    2
41284    2
41285    2
41286    2
41287    1
41288    2
Name: PostTypeId, dtype: int64

In [206]:
total_travel_df['accepted'] = total_travel_df['PostTypeId'] - 1 

In [129]:
cvec2 = CountVectorizer(stop_words = 'english', ngram_range=(1,3))

In [207]:
X2 = cvec2.fit_transform(total_travel_df['Text'])

In [208]:
y2 =  total_travel_df['accepted']

In [209]:
y2.shape, X2.shape

((41289,), (41289, 977288))

In [143]:
#from sklearn.utils import shuffle
#X_shuf, Y_shuf = shuffle(X2, y2)

In [213]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.4)

In [214]:
X_train2.shape, y_train2.shape

((24773, 977288), (24773,))

In [215]:
logreg2 = LogisticRegression()

In [216]:
logreg2.fit(X_train2, y_train2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3. What is the score of a post?

---

**3.1 Build a model that predicts the score of a post.**

- This is a regression problem now. 
- You can and should be predicting score for both "question" and "answer" posts, so keep them both in your dataset.
- Again, use any techniques that you think will get you the best model.

**3.2 Evaluate the performance of your model with cross-validation and report the results.**

**3.3 What is important for determining the score of a post, if anything?**


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 4. How many views does a post have?

---

**4.1 Build a model that predicts the number of views a post has.**

- This is another regression problem. 
- Predict the views for all posts, not just the "answer" posts.

**4.2 Evaluate the performance of your model with cross-validation and report the results.**

**4.3 What is important for the number of views a post has, if anything?**

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5. Build a pipeline or other code to automate evaluation of your models on the test data.

---

Now that you've constructed your three predictive models, build a pipeline or code that can easily load up the raw testing data and evaluate your models on it.

The testing data that is held out is in the same raw format as the training data you have. _Any cleaning and preprocessing that you did on the training data will need to be done on the testing data as well!_

This is a good opportunity to practice building pipelines, but you're not required to. Custom functions and classes are fine as long as they are able to process and test the new data.
